### 동적 스크레이핑으로 모든 도서의 링크만 가져오기
- 유효하지 않은 3개의 링크는 추후 json 파일 편집으로 제거함 ( 링크 대신 javascript:void(0); 이 기록된 경우)

In [89]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import json

# WebDriver 설정
driver = webdriver.Chrome()
driver.get('https://www.yes24.com/Product/Category/Display/001001047001')
time.sleep(2)  # 페이지 로드 대기

# 스크롤 다운 함수
def scroll_down(driver, scroll_count):
    for _ in range(scroll_count):
        driver.execute_script("window.scrollBy(0, 1000);")  # 스크롤 내리기
        # time.sleep(0.5)  # 대기

# 메인 페이지 스크롤
main_scroll_cnts = 9

# 페이지 번호 클릭 -> 이동
Total_pages_num = 1745
book_links = []

for page_num in range(1, Total_pages_num+1):
    try:
        scroll_down(driver, main_scroll_cnts)
        if page_num % 10 == 1:
            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong')))
            page_link.click()
            print(page_num)
            time.sleep(1)
    
        elif 2 <= page_num < 10:
            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f'#categoryProductContentsWrap .newGoodsPagen div div a:nth-child({page_num})')))
            page_link.click()
            print(page_num)
            time.sleep(1)

        elif page_num % 10 == 0:

            page_link = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong')))
            page_link.click()
            print(page_num)
            time.sleep(1)

            scroll_down(driver, main_scroll_cnts)

            next_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR,'#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next')))
            next_button.click()
            print(f"{page_num} 'next button'")
            time.sleep(1)

        else:
            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f'#categoryProductContentsWrap .newGoodsPagen div div a:nth-child({ (page_num % 10) + 2 })')))
            page_link.click()
            print(page_num)
            time.sleep(1)

        # 책 링크 수집
        books = driver.find_elements(By.CSS_SELECTOR, 'a.gd_name')  # 책 제목 링크
        for book in books:
            book_links.append(book.get_attribute('href'))
        print(f"수집한 링크 개수: {len(book_links)}")

    except Exception as e:
        print(f"오류 발생 (페이지 번호 {page_num}): {e}")
        continue

### 페이지 html 리스트#1_10
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(2)
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(10)
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.end

#11~20
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.first
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.prev
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(4)
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(5) .....
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(11)
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong         ## 20번 페이지


#21~30
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.first
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.prev
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(4)
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(5)

#next button
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next

    



1
수집한 링크 개수: 24
2
수집한 링크 개수: 48
3
수집한 링크 개수: 72
4
수집한 링크 개수: 96
5
수집한 링크 개수: 120
6
수집한 링크 개수: 144
7
수집한 링크 개수: 168
8
수집한 링크 개수: 192
9
수집한 링크 개수: 216
10
10 'next button'
수집한 링크 개수: 240
11
수집한 링크 개수: 264
12
수집한 링크 개수: 288
13
수집한 링크 개수: 312
14
수집한 링크 개수: 336
15
수집한 링크 개수: 360
16
수집한 링크 개수: 384
17
수집한 링크 개수: 408
18
수집한 링크 개수: 432
19
수집한 링크 개수: 456
20
20 'next button'
수집한 링크 개수: 480
21
수집한 링크 개수: 504
22
수집한 링크 개수: 528
23
수집한 링크 개수: 552
24
수집한 링크 개수: 576
25
수집한 링크 개수: 600
26
수집한 링크 개수: 624
27
수집한 링크 개수: 648
28
수집한 링크 개수: 672
29
수집한 링크 개수: 696
30
30 'next button'
수집한 링크 개수: 720
31
수집한 링크 개수: 744
32
수집한 링크 개수: 768
33
수집한 링크 개수: 792
34
수집한 링크 개수: 816
35
수집한 링크 개수: 840
36
수집한 링크 개수: 864
37
수집한 링크 개수: 888
38
수집한 링크 개수: 912
39
수집한 링크 개수: 936
40
40 'next button'
수집한 링크 개수: 960
오류 발생 (페이지 번호 41): Message: element click intercepted: Element <strong class="num">...</strong> is not clickable at point (949, 421). Other element would receive the click: <div id="yesComLoading" style="z-index:199999"></div

In [90]:
# 페이지 1745 / 수집한 링크 개수: 40872
len(book_links)

40872

In [92]:
with open('all_book_links.json', 'w', encoding='utf-8') as f:
    json.dump(book_links, f, ensure_ascii=False, indent=4)
print("모든 에세이 도서의 링크 수집 완료!")

모든 에세이 도서의 링크 수집 완료!


## 정적 웹스크레이핑

In [ ]:
import json
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup


with open('all_book_links.json', 'r', encoding='utf-8') as file:
    all_book_links = json.load(file)

all_book_data = []
invalid_url_cnt = 0
invalid_url_idx = []
img_url_error_count = 0
img_url_error_idx = []
isbn13_error_count = 0
isbn13_error_idx = []
date_error_count = 0
date_error_idx = []


for idx, link in enumerate(all_book_links[15370:15380]):
    
    if not link.startswith('http'):
        print(f"Invalid URL skipped : {link}")
        invalid_url_cnt += 1
        invalid_url_idx.append(idx)
        continue

    response = requests.get(link)
    
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    ## 줄바꿈으로 부제도 표현됨.
    titles = soup.select('h2.gd_name')
    title_text = '\n'.join(set([title.get_text(strip=True) for title in titles]))

    # 도서별 책소개의 css는 총 2종류이다
    descriptions = soup.select('#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt > div')
    if len(descriptions) == 0:
        descriptions = soup.select('#infoset_introduce > div.infoSetCont_wrap > table > tbody > tr > td > div > div')
    descriptions_text = '\n'.join([desc.get_text(strip=True) for desc in descriptions])

    key_sentences = soup.select('#infoset_inBook > div.infoSetCont_wrap > div.infoWrap_txt > div')
    key_sentences_text = '\n'.join([key_sentence.get_text(strip=True) for key_sentence in key_sentences])

    if descriptions_text and key_sentences_text:
        authors = soup.select('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a')
        authors_text = ', '.join([author.get_text(strip=True) for author in authors])

        publishers = soup.select('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a')
        publishers_text = ', '.join([publisher.get_text(strip=True) for publisher in publishers])

        isbn13 = soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr')[-2]
        if isbn13:
            isbn13_text = isbn13.get_text(strip=True)
            isbn13_num_only = int(re.sub(r'[^0-9]', '', isbn13_text))
            try:
                isbn13_int = int(isbn13_num_only)
            except ValueError:
                print(f"Invalid ISBN data: {isbn13_num_only}")
                isbn13_int = isbn13_text
                isbn13_error_count += 1
                isbn13_error_idx.append(idx)
        
        img_url = soup.select_one('img.gImg')['src']
        if img_url is None:
            img_url_error_count += 1
            img_url_error_idx.append(idx)

        publication_date = soup.select_one('span.gd_date')
        if publication_date is not None:
            publication_date_text = publication_date.text
            parsed_date = datetime.strptime(publication_date_text, "%Y년 %m월 %d일")
            formatted_date = parsed_date.strftime("%Y-%m-%d")
        else:
            print(f"Invalid Publication_date: {idx}th book")
            formatted_date = None
            date_error_count += 1
            date_error_idx.append(idx)


        all_book_data.append({
                'title': title_text,
                'description': descriptions_text,
                'key_sentences': key_sentences_text,
                'author': authors_text,
                'publisher': publishers_text,
                'isbn13': isbn13_int,
                'img_url': img_url,
                'publication_date': formatted_date
            })
        
    print(f'{idx}-th completed')
    

with open('all_book_data_ver3.json', 'w', encoding='utf-8') as f:
    json.dump(all_book_data, f, ensure_ascii=False, indent=4)


In [ ]:
print(len(all_book_data))

print(f"{invalid_url_cnt = }")
print(f'{invalid_url_idx = }')
print(f'{img_url_error_count = }')
print(f"{img_url_error_idx = }")
print(f"{isbn13_error_count = }")
print(f"{isbn13_error_idx = }")
print(f"{date_error_count = }")
print(f"{date_error_idx = }")
